In [2]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 데이터 가져오기

In [3]:
import pandas as pd
import pandasql as ps

df = pd.read_csv('/Volumes/DATA/mbc/ai_x/download/shareData/부동산_250213/최종전국평당분양가격(결측치제외).csv', 
                 encoding='cp949')
df.head()

,지역명,연도,월,평당분양가격
0,서울,2013,12,18189.0
1,부산,2013,12,8111.0
2,대구,2013,12,8080.0
3,인천,2013,12,10204.0
4,광주,2013,12,6098.0


# 2. 지역명의 라벨 인코딩

- 지역명을 라벨인코딩한 지역명2 (LabelEncoder 이용)

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['지역명2'] = le.fit_transform(df['지역명'])
# le.fit_transform(df['지역명']).tolist()
# df['지역명2'] =  = pd.Series(le.fit_transform(df['지역명']))
df.sample()
print(df['지역명'].value_counts())
print(df['지역명2'].value_counts())

서울    128
강원    128
경남    128
경북    128
전남    128
전북    128
충남    128
충북    128
세종    128
부산    128
경기    128
울산    128
대전    128
광주    128
인천    128
대구    128
제주    128
Name: 지역명, dtype: int64
8     128
0     128
2     128
3     128
12    128
13    128
15    128
16    128
9     128
7     128
1     128
10    128
6     128
4     128
11    128
5     128
14    128
Name: 지역명2, dtype: int64


# 3. nomalization 스케일 조정
- 입력변수(지역명2, 연도, 월)와 타켓변수(평당분양가격) 따로 스케일 조정(Minmaxscaler 이용)
- 지역명2n,연도n, 월n, 평당분양가격n 필드 추가

In [5]:
df[['지역명2','연도','월','평당분양가격']].sample()

,지역명2,연도,월,평당분양가격
614,1,2017,1,11728.2


In [6]:
# 전체내용 : df
# 독립변수를 넘파이배열로 : df[['지역명2','연도','월']].values
# 독립변수를 넘파이배열로 : df[['평당분양가격']].values

X_data = df[['지역명2','연도','월']].values
Y_data = df[['평당분양가격']].values

print(X_data.shape)
print(Y_data.shape)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

df[['지역명2n','연도n','월n']] = x_scaler.fit_transform(X_data)
df[['평당분양가격n']] = y_scaler.fit_transform(Y_data)

df.head()

(2176, 3)
(2176, 1)


,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757


# 4. standarization 스케일 조정

- 입력변수(지역명2, 연도, 월)와 타켓변수(평당분양가격) 따로 스케일 조정(Standarization 이용)
- 지역명2s, 연도s, 월s, 평당분양가격s 필드 추가

In [7]:
from sklearn.preprocessing import StandardScaler

X_data = df[['지역명2','연도','월']].values
Y_data = df[['평당분양가격']].values

print(X_data.shape)
print(Y_data.shape)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

x_scaler = StandardScaler()
y_scaler = StandardScaler()

df[['지역명2s','연도s','월s']] = x_scaler.fit_transform(X_data)
df[['평당분양가격s']] = y_scaler.fit_transform(Y_data)

df.head()

(2176, 3)
(2176, 1)


,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.167561
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.727778
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.733608
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334154
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.106357


# 5. 지역명을 원핫인코딩

In [8]:
from tensorflow.keras.utils import to_categorical # 분류분석시 원핫인코딩

to_categorical(df['지역명2'])

2025-06-11 10:33:47.872923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [9]:
# 지역명, 지역명2
# df
loc_info = df[['지역명','지역명2']].head(17).sort_values(by='지역명2')
loc_column_names = loc_info['지역명'].to_list()
print(loc_column_names)

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [11]:
import numpy as np

df[loc_column_names] = to_categorical(df['지역명2']).astype(np.int16)
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,...,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,...,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,...,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,...,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,...,0,0,0,0,0,0,0,0,0,0


In [14]:
pd.options.display.max_columns = 30 # 최대 출력가능한 데이터프레임 열수
df.head()

,지역명,연도,월,평당분양가격,지역명2,지역명2n,연도n,월n,평당분양가격n,지역명2s,연도s,월s,평당분양가격s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.167561,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.727778,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.733608,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334154,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.106357,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
import pandas as pd
from pandasql import sqldf

# 테스트용 DataFrame 생성
df = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'age': [25, 30, 35, 40],
    'city': ['Seoul', 'Busan', 'Seoul', 'Incheon']
})

# SQL 쿼리 실행 함수 (로컬 스코프 사용)
pysqldf = lambda q: sqldf(q, globals())

# SQL 쿼리
query = "SELECT name, age FROM df WHERE city = 'Seoul'"
result = pysqldf(query)

print(result)

      name  age
0    Alice   25
1  Charlie   35
